In [1]:
import numpy as np
from pyspark import SparkContext

sc = SparkContext("local", "pyspark")

Implementing Matrix Multiplication using Spark RDD's **transformation** and **action** operations

Initializing two random matricies of dimensions (2, 3) and (3, 2)

In [2]:
A = np.random.rand(2, 3)
B = np.random.rand(3, 2)

In [3]:
A

array([[ 0.17899335,  0.80960283,  0.67597551],
       [ 0.66028443,  0.5952246 ,  0.32610798]])

In [4]:
B

array([[ 0.39127727,  0.06911717],
       [ 0.87711723,  0.70815121],
       [ 0.91027243,  0.8023777 ]])

Their product is

In [5]:
A.dot(B)

array([[ 1.39547449,  1.12808041],
       [ 1.07728314,  0.72880779]])

Turn them into Spark RDD:

In [6]:
rddA = sc.parallelize(list(enumerate(A.T)))
rddA = rddA.flatMapValues(lambda x: list(enumerate(x)))

rddB = sc.parallelize(list(enumerate(B)))
rddB = rddB.flatMapValues(lambda x: list(enumerate(x)))

This is what they look like:

In [7]:
rddA.collect()

[(0, (0, 0.17899334850433291)),
 (0, (1, 0.66028443390786606)),
 (1, (0, 0.80960283194218141)),
 (1, (1, 0.59522459696916552)),
 (2, (0, 0.67597551027170466)),
 (2, (1, 0.32610798183646017))]

In [8]:
rddB.collect()

[(0, (0, 0.39127726636455873)),
 (0, (1, 0.069117174413648286)),
 (1, (0, 0.87711723433813138)),
 (1, (1, 0.70815121072590004)),
 (2, (0, 0.91027242758994387)),
 (2, (1, 0.80237769866001274))]

Join them on A's column index (or B's row index), and do dot-product on A's row vectors with B's column vectors:

In [9]:
C = rddA.join(rddB).map(lambda x: ((x[1][0][0], x[1][1][0]),
                                   x[1][0][1] * x[1][1][1])).reduceByKey(lambda x, y: x + y)

C.collect()

[((0, 0), 1.3954744936920349),
 ((1, 1), 0.72880778535917756),
 ((0, 1), 1.1280804144167682),
 ((1, 0), 1.0772831449088951)]

Finally, clean up the result **C**:

In [10]:
C = C.map(lambda x: (x[0][0],(x[0][1], x[1]))).groupByKey()
C = C.mapValues(list).mapValues(lambda x: sorted(x, key=lambda y: y[0]))
C = C.mapValues(lambda x: zip(*x)[1])

C = np.array(C.sortByKey().map(lambda x: np.array(x[1])).collect())

In [11]:
C

array([[ 1.39547449,  1.12808041],
       [ 1.07728314,  0.72880779]])

which equals A.dot(B):

In [12]:
A.dot(B)

array([[ 1.39547449,  1.12808041],
       [ 1.07728314,  0.72880779]])

<img src="MatrixMulSparkRDD1.png">

<img src="MatrixMulSparkRDD2.png">

<img src="MatrixMulSparkRDD3.png">

<img src="MatrixMulSparkRDD4.png">